In [50]:
# install.packages('StatMatch')
# install.packages('reshape')
# install.packages('ade4')
# install.packagesckages('stringi')
# install.packages('Hmisc')""
# install.packages("tidyr")


Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [1]:
library(StatMatch)
library(reshape)
library(ade4)
library(Hmisc)
library(tidyr)

Loading required package: proxy


Attaching package: ‘proxy’


The following objects are masked from ‘package:stats’:

    as.dist, dist


The following object is masked from ‘package:base’:

    as.matrix


Loading required package: clue

Loading required package: survey

Loading required package: grid

Loading required package: Matrix

Loading required package: survival


Attaching package: ‘survey’


The following object is masked from ‘package:graphics’:

    dotchart


Loading required package: RANN

Loading required package: lpSolve


Attaching package: ‘reshape’


The following object is masked from ‘package:Matrix’:

    expand


Loading required package: lattice

Loading required package: Formula

Loading required package: ggplot2


Attaching package: ‘Hmisc’


The following object is masked from ‘package:survey’:

    deff


The following objects are masked from ‘package:base’:

    format.pval, units



Attaching package: ‘tidyr’


The following objects are masked from ‘pack

In [2]:
# Read in data
recs = read.csv("./data/recs2015_public_v4.csv")
pums<- read.csv("./data/ss15hpa.csv")

In [3]:
# reformating common variables in recs
recs$BTUOTHER <- recs$TOTALBTU - recs$BTUEL - recs$BTUNG

recs$x_hhtype <- factor(recs$TYPEHUQ, labels = c(" mobile home", " single family detached", " single family attached", " apartment in building with 2-4 units", " apartment in building with 5+ units"))
recs$x_tenure <- factor(recs$KOWNRENT, labels = c(" owned by someone in the household", " rented", " occupied without payment of rent"))

recs$x_yearbuilt <- factor(recs$YEARMADERANGE,
                        labels = c(" 1950 or earlier", " 1950-1959", " 1960-1969", " 1970-1979",
                                   " 1980-1989"," 1990-1999", " 2000-2009", " 2010-2015"), order = TRUE)

recs$x_mvin <- cut(recs$OCCUPYYRANGE, breaks = c(-Inf, 4, 5, 6,7,Inf), labels = c(" 30 years or more", " 20-29 years", " 10-19 years", " 5-9 years", " 4 or less years"), ordered = TRUE)

recs$BEDROOMS[recs$BEDROOMS<0] <- NA
recs$x_bedrooms <- recs$BEDROOMS

recs$x_totrooms <- recs$TOTROOMS

recs$x_heatfuel <- cut(recs$FUELHEAT, breaks = c(-Inf, 0, 1, 2, 4,5,7, Inf), labels = c(" No fuel used", " natural gas", " propange/LPG", " fuel oil or Kerosene", " electricity", " wood", " other fuel"))
recs$x_heatfuel <- recs$FUELHEAT
recs$x_heatfuel[recs$x_heatfuel == 2] <- 8
recs$x_heatfuel[recs$x_heatfuel == 4] <- 8
recs$x_heatfuel[recs$x_heatfuel == 7] <- 8
recs$x_heatfuel[recs$x_heatfuel == 0] <- 8
recs$x_heatfuel <- cut(recs$x_heatfuel, breaks = c(-Inf, 1, 5,Inf), labels = c( " natural gas",  " electricity", " other fuel"))

recs$x_hhsize <- recs$NHSLDMEM
recs$x_hhincome <- factor(recs$MONEYPY, labels = c("Less than $20,000",
"$20,000 - $39,999",
"$40,000 - $59,999",
"$60,000 to $79,999",
"$80,000 to $99,999",
"$100,000 to $119,999",
"$120,000 to $139,999",
"$140,000 or more"),ordered = TRUE)
recs$x_yearel <- recs$DOLLAREL

recs$HDD65[recs$HDD65 ==0 ] = mean(recs$HDD65)

recs$x_yearng <- recs$DOLLARNG/recs$HDD65*374 #https://www.eia.gov/totalenergy/data/annual/showtext.php?t=ptb0107
recs$x_yearother <- recs$TOTALDOL - recs$DOLLAREL - recs$DOLLARNG


In [4]:
# reformating common variables in pums
pums<- pums[,1:82] #get rid of the weights
pums$x_hhtype <- cut(pums$BLD, breaks= c(-Inf, 1,2,3,5,Inf), labels = c(" mobile home", " single family detached", " single family attached", " apartment in building with 2-4 units", " apartment in building with 5+ units"))
pums$x_tenure <- cut(pums$TEN, breaks = c(-Inf, 2, 3, Inf), labels = c(" owned by someone in the household", " rented", " occupied without payment of rent"))

pums$x_yearbuilt <- cut(replace_na(pums$YBL, 20), 
                        breaks = c(-Inf, 2, 3, 4, 5, 6, 7, 13, 19, Inf),
                        labels = c(" 1950 or earlier", " 1950-1959", " 1960-1969", " 1970-1979",
                                   " 1980-1989"," 1990-1999", " 2000-2009", " 2010-2015", 'UNK'),
                        ordered = TRUE)


pums$x_mvin <- cut(pums$MV, breaks = c(-Inf, 3, 4, 5, 6,Inf), labels = c(" 4 or less years"," 5-9 years" , " 10-19 years",  " 20-29 years"," 30 years or more" ), ordered = TRUE)
pums$x_bedrooms <- pums$BDSP
pums$x_totrooms <- pums$RMSP
pums$x_heatfuel <- pums$HFL
pums$x_heatfuel[pums$x_heatfuel==5] <- 8
pums$x_heatfuel[pums$x_heatfuel==2] <- 8
pums$x_heatfuel[pums$x_heatfuel==4] <- 8
pums$x_heatfuel[pums$x_heatfuel==6] <- 8
pums$x_heatfuel[pums$x_heatfuel==9] <- 8
pums$x_heatfuel <- cut(pums$x_heatfuel, breaks = c(-Inf, 1,3, Inf), labels = c( " natural gas", " electricity",  " other fuel"))
pums$x_hhsize <- pums$NP
pums$x_hhincome <- addNA(cut(pums$HINCP,
    breaks = c(-Inf, 20000, 40000, 60000, 80000, 100000, 120000, 140000, Inf),
    labels = c("Less than $20,000",
                "$20,000 to $39,999",
                "$40,000 to $59,999",
                "$60,000 to $79,999",
                "$80,000 to $99,999",
                "$100,000 to $119,999",
                "$120,000 to $139,999",
                "$140,000 or more"),ordered = TRUE))

pums$x_yearel <- pums$ELEP*12
pums$x_yearel[pums$x_yearel==24] <- 0
pums$x_yearel[pums$x_yearel==12] <- 0


pums$x_yearng <- pums$GASP
pums$x_yearng[pums$x_yearng==1] <- 0
pums$x_yearng[pums$x_yearng==2] <- 0
pums$x_yearng[pums$x_yearng==3] <- 0

pums$x_yearother <- pums$FULP
pums$x_yearother[pums$x_yearother==1] <- 0
pums$x_yearother[pums$x_yearother==2] <- 0

spearman2(TOTALBTU~x_hhtype+x_tenure+x_yearbuilt+x_mvin+x_bedrooms+x_totrooms+x_heatfuel+x_hhsize+ x_hhincome+ x_yearel+ x_yearng+ x_yearother, p =2, data = recs)
spearman2(BTUEL~x_hhtype+x_tenure+x_yearbuilt+x_mvin+x_bedrooms+x_totrooms+x_heatfuel+x_hhsize+ x_hhincome+ x_yearel+ x_yearng+ x_yearother, p =2, data = recs)
spearman2(BTUNG~x_hhtype+x_tenure+x_yearbuilt+x_mvin+x_bedrooms+x_totrooms+x_heatfuel+x_hhsize+ x_hhincome+ x_yearel+ x_yearng+ x_yearother, p =2, data = recs)
spearman2(BTUOTHER~x_hhtype+x_tenure+x_yearbuilt+x_mvin+x_bedrooms+x_totrooms+x_heatfuel+x_hhsize+ x_hhincome+ x_yearel+ x_yearng+ x_yearother, p =2, data = recs)


Spearman rho^2    Response variable:TOTALBTU

             rho2       F df1  df2 P Adjusted rho2    n
x_hhtype    0.325  684.34   4 5681 0         0.325 5686
x_tenure    0.192  675.21   2 5683 0         0.192 5686
x_yearbuilt 0.030   25.40   7 5678 0         0.029 5686
x_mvin      0.083  128.31   4 5681 0         0.082 5686
x_bedrooms  0.314 1298.66   2 5683 0         0.313 5686
x_totrooms  0.356 1570.29   2 5683 0         0.356 5686
x_heatfuel  0.117  374.73   2 5683 0         0.116 5686
x_hhsize    0.100  316.67   2 5683 0         0.100 5686
x_hhincome  0.115  105.84   7 5678 0         0.114 5686
x_yearel    0.223  816.38   2 5683 0         0.223 5686
x_yearng    0.155  519.88   2 5683 0         0.154 5686
x_yearother 0.166  565.16   2 5683 0         0.166 5686


Spearman rho^2    Response variable:BTUEL

             rho2        F df1  df2 P Adjusted rho2    n
x_hhtype    0.193   340.69   4 5681 0         0.193 5686
x_tenure    0.100   314.37   2 5683 0         0.099 5686
x_yearbuilt 0.041    35.02   7 5678 0         0.040 5686
x_mvin      0.036    53.35   4 5681 0         0.036 5686
x_bedrooms  0.184   642.85   2 5683 0         0.184 5686
x_totrooms  0.189   660.29   2 5683 0         0.188 5686
x_heatfuel  0.098   309.80   2 5683 0         0.098 5686
x_hhsize    0.113   360.24   2 5683 0         0.112 5686
x_hhincome  0.052    44.80   7 5678 0         0.051 5686
x_yearel    0.831 13967.48   2 5683 0         0.831 5686
x_yearng    0.172   591.81   2 5683 0         0.172 5686
x_yearother 0.004    10.24   2 5683 0         0.003 5686


Spearman rho^2    Response variable:BTUNG

             rho2        F df1  df2 P Adjusted rho2    n
x_hhtype    0.062    93.58   4 5681 0         0.061 5686
x_tenure    0.027    79.66   2 5683 0         0.027 5686
x_yearbuilt 0.044    37.10   7 5678 0         0.043 5686
x_mvin      0.012    17.38   4 5681 0         0.011 5686
x_bedrooms  0.064   193.38   2 5683 0         0.063 5686
x_totrooms  0.070   212.93   2 5683 0         0.069 5686
x_heatfuel  0.673  5838.42   2 5683 0         0.673 5686
x_hhsize    0.011    31.66   2 5683 0         0.011 5686
x_hhincome  0.037    30.93   7 5678 0         0.036 5686
x_yearel    0.031    89.62   2 5683 0         0.030 5686
x_yearng    0.828 13634.32   2 5683 0         0.827 5686
x_yearother 0.098   307.29   2 5683 0         0.097 5686


Spearman rho^2    Response variable:BTUOTHER

             rho2       F df1  df2      P Adjusted rho2    n
x_hhtype    0.014   19.58   4 5681 0.0000         0.013 5686
x_tenure    0.009   25.47   2 5683 0.0000         0.009 5686
x_yearbuilt 0.002    1.50   7 5678 0.1613         0.001 5686
x_mvin      0.009   12.63   4 5681 0.0000         0.008 5686
x_bedrooms  0.002    6.59   2 5683 0.0014         0.002 5686
x_totrooms  0.004   12.80   2 5683 0.0000         0.004 5686
x_heatfuel  0.140  460.91   2 5683 0.0000         0.139 5686
x_hhsize    0.001    4.17   2 5683 0.0154         0.001 5686
x_hhincome  0.001    1.17   7 5678 0.3150         0.000 5686
x_yearel    0.007   18.79   2 5683 0.0000         0.006 5686
x_yearng    0.072  218.82   2 5683 0.0000         0.071 5686
x_yearother 0.310 1278.66   2 5683 0.0000         0.310 5686

In [5]:
# drop NA values and standardize common variables
pums <- pums[complete.cases(pums[,c("x_hhtype", "x_heatfuel", "x_totrooms", "x_yearel", "x_yearng", "x_yearother")]),]

merge_totrooms <- c(pums$x_totrooms, recs$x_totrooms)
sd <- sd(merge_totrooms, na.rm = TRUE)
mean <- mean(merge_totrooms, na.rm = TRUE)
pums$x_totrooms_std <- (pums$x_totrooms - mean)/sd
recs$x_totrooms_std <- (recs$x_totrooms - mean)/sd

merge_yearel <- c(pums$x_yearel, recs$x_yearel)
sd <- sd(merge_yearel, na.rm = TRUE)
mean <- mean(merge_yearel, na.rm = TRUE)
pums$x_yearel_std <- (pums$x_yearel - mean)/sd
recs$x_yearel_std <- (recs$x_yearel - mean)/sd

merge_yearng <- c(pums$x_yearng, recs$x_yearng)
sd <- sd(merge_yearng, na.rm = TRUE)
mean <- mean(merge_yearng, na.rm = TRUE)
pums$x_yearng_std <- (pums$x_yearng - mean)/sd
recs$x_yearng_std <- (recs$x_yearng - mean)/sd

merge_yearother <- c(pums$x_yearother, recs$x_yearother)
sd <- sd(merge_yearother, na.rm = TRUE)
mean <- mean(merge_yearother, na.rm = TRUE)
pums$x_yearother_std <- (pums$x_yearother - mean)/sd
recs$x_yearother_std <- (recs$x_yearother - mean)/sd

merge_hhsize <- c(pums$x_hhsize, recs$x_hhsize)
sd <- sd(merge_hhsize, na.rm = TRUE)
mean <- mean(merge_hhsize, na.rm = TRUE)
pums$x_hhsize_std <- (pums$x_hhsize - mean)/sd
recs$x_hhsize_std <- (recs$x_hhsize - mean)/sd


In [6]:
# statmatch electricity BTU
group.v <- c("x_hhtype")
X.mtc <- c("x_totrooms_std", "x_yearel_std")
X.join <- c("SERIALNO","x_totrooms","x_yearel","x_yearng", "x_hhtype", "x_heatfuel")

out.el <- NND.hotdeck(data.rec = recs, data.don = pums, match.vars = X.mtc, don.class = group.v, constrained = TRUE )
elbtu <- create.fused(data.rec = recs, data.don = pums, mtc.ids = out.el$mtc.ids, z.vars=X.join)
elbtu$dist <- out.el$dist.rd
summary(elbtu$dist)
elbtu<- elbtu[elbtu$dist<1,] #cut off distance above 1

elbtu <- elbtu[,c("SERIALNO","BTUEL")]
pums_el_final <- merge(pums, elbtu, by="SERIALNO", all.x = TRUE)
el_training <- pums_el_final[complete.cases(pums_el_final[,"BTUEL"]),]
el_imputing <- pums_el_final[is.na(pums_el_final$"BTUEL"),]


All the categorical matching variables in rec and don 
 data.frames, if present are recoded into dummies


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.00000 0.01382 0.02664 0.02968 0.03969 1.31634 

In [7]:
# prepare electric outputs for machine learning
el_training <- el_training[,-83:-99]
el_imputing <- el_imputing[,-83:-100]
el_training[is.na(el_training)] <- 0
el_imputing[is.na(el_imputing)] <- 0

write.csv(el_training, "output/el_training.csv", row.names =FALSE)
write.csv(el_imputing, "output/el_imputing.csv", row.names =FALSE)


In [8]:
# statmatch natural gas BTU
group.v <- c("x_heatfuel")
X.mtc <- c("x_yearng_std")
X.join <- c("SERIALNO","x_yearng", "x_heatfuel")


In [9]:
out.ng <- NND.hotdeck(data.rec = recs, data.don = pums, match.vars = X.mtc, don.class = group.v, constrained = TRUE )
ngbtu <- create.fused(data.rec = recs, data.don = pums, mtc.ids = out.ng$mtc.ids, z.vars=X.join)
ngbtu$dist <- out.ng$dist.rd
ngbtu<- ngbtu[ngbtu$dist<1,]

ngbtu <- ngbtu[,c("SERIALNO","BTUNG")]
pums_ng_final <- merge(pums, ngbtu, by="SERIALNO", all.x = TRUE)
ng_training <- pums_ng_final[complete.cases(pums_ng_final[,"BTUNG"]),]
ng_imputing <- pums_ng_final[is.na(pums_ng_final$"BTUNG"),]

ng_training <- ng_training[,-83:-99]
ng_imputing <- ng_imputing[,-83:-100]
ng_training[is.na(ng_training)] <- 0
ng_imputing[is.na(ng_imputing)] <- 0


All the categorical matching variables in rec and don 
 data.frames, if present are recoded into dummies


In [10]:
write.csv(ng_training, "output/ng_training.csv", row.names =FALSE)
write.csv(ng_imputing, "output/ng_imputing.csv", row.names =FALSE)

In [11]:
# statmatch other BTU
group.v <- c("x_heatfuel")
X.mtc <- c("x_yearother_std")
X.join <- c("SERIALNO","x_yearother", "x_heatfuel")

out.other<- NND.hotdeck(data.rec = recs, data.don = pums, match.vars = X.mtc, don.class = group.v, constrained = TRUE )
obtu <- create.fused(data.rec = recs, data.don = pums, mtc.ids = out.other$mtc.ids, z.vars=X.join)
obtu$dist <- out.other$dist.rd
obtu <- obtu[obtu$dist<1, ]
obtu <- obtu[,c("SERIALNO","BTUOTHER")]
pums_other_final <- merge(pums, obtu, by="SERIALNO", all.x = TRUE)

other_training <- pums_other_final[complete.cases(pums_other_final[,"BTUOTHER"]),]
other_imputing <- pums_other_final[is.na(pums_other_final$"BTUOTHER"),]
other_training <- other_training[,-83:-99]
other_imputing <- other_imputing[,-83:-100]
other_training[is.na(other_training)] <- 0
other_imputing[is.na(other_imputing)] <- 0

write.csv(other_training, "output/other_training.csv", row.names =FALSE)
write.csv(other_imputing, "output/other_imputing.csv", row.names =FALSE)




All the categorical matching variables in rec and don 
 data.frames, if present are recoded into dummies
